IMPORT PACKAGES

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
import datetime
import gc
import xgboost as xgb
from sklearn import preprocessing
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error

from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
import pickle

In [2]:
train = pd.read_csv('./Downloads/Train (2).csv')
test = pd.read_csv('./Downloads/Test (2).csv')

GENERATING NEW FEATURES FOR FUTURE DAYS

In [3]:
empty_list = ['temperature_2m_above_ground','precipitable_water_entire_atmosphere']


for i in empty_list:
    train[i+'_new_'+str(2)] = train[i].shift(periods = 2)
    train[i+'_new_'+str(3)] = train[i].shift(periods = 3)
    train[i+'_new_'+str(4)] = train[i].shift(periods = 4)
    test[i+'_new_'+str(2)] = test[i].shift(periods = 2)
    test[i+'_new_'+str(3)] = test[i].shift(periods = 3)
    test[i+'_new_'+str(4)] = test[i].shift(periods = 4)
    

In [4]:
features_corr_ = ['L3_NO2_NO2_column_number_density','L3_O3_O3_column_number_density','L3_HCHO_tropospheric_HCHO_column_number_density','L3_CO_CO_column_number_density','L3_CLOUD_surface_albedo','L3_CLOUD_cloud_optical_depth']

for i in features_corr_:
    train[i+'_new_'+str(2)] = train[i].shift(periods = 2)
    train[i+'_new_'+str(3)] = train[i].shift(periods = 3)
    test[i+'_new_'+str(2)] = test[i].shift(periods = 2)
    test[i+'_new_'+str(3)] = test[i].shift(periods = 3)

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30557 entries, 0 to 30556
Data columns (total 100 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Place_ID X Date                                        30557 non-null  object 
 1   Date                                                   30557 non-null  object 
 2   Place_ID                                               30557 non-null  object 
 3   target                                                 30557 non-null  float64
 4   target_min                                             30557 non-null  float64
 5   target_max                                             30557 non-null  float64
 6   target_variance                                        30557 non-null  float64
 7   target_count                                           30557 non-null  int64  
 8   precipitable_water_entire_atmosphere         

In [ ]:
USING PREVIOUS DATA

In [6]:
def func(df,list_,k):
    y_day = [np.NaN]
    s = df[list_[k]][0:len(df) - 1]
    for i in s:                      
        y_day.append(i)             
    df[list_[k] + '_yday_'] = y_day

In [7]:
def func1(df1,list1_,k):
    y_day2 = [np.NaN]
    s1 = df1[list1_[k]][0:len(df1) - 1]
    for j in s1:
        y_day2.append(j)
    df1[list1_[k] + '_yday_'] = y_day2

In [8]:
emma_ = ['precipitable_water_entire_atmosphere','temperature_2m_above_ground','relative_humidity_2m_above_ground']

In [9]:
func(train,emma_,0), func1(test,emma_,0)

(None, None)

In [10]:
func(train,emma_,1), func1(test,emma_,1)

(None, None)

In [11]:
func(train,emma_,2), func1(test,emma_,2)

(None, None)

In [12]:
train.head()

,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,...,L3_HCHO_tropospheric_HCHO_column_number_density_new_3,L3_CO_CO_column_number_density_new_2,L3_CO_CO_column_number_density_new_3,L3_CLOUD_surface_albedo_new_2,L3_CLOUD_surface_albedo_new_3,L3_CLOUD_cloud_optical_depth_new_2,L3_CLOUD_cloud_optical_depth_new_3,precipitable_water_entire_atmosphere_yday_,temperature_2m_above_ground_yday_,relative_humidity_2m_above_ground_yday_
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92,11.000000,60.200001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91,14.600000,48.799999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.000000,18.516840,60.200001
2,010Q650 X 2020-01-04,2020-01-04,010Q650,24.0,8.0,56.0,1181.96,96,16.400000,33.400002,...,NaN,0.021080,NaN,NaN,NaN,NaN,NaN,14.600000,22.546533,48.799999
3,010Q650 X 2020-01-05,2020-01-05,010Q650,49.0,10.0,55.0,1113.67,96,6.911948,21.300001,...,0.000064,0.022017,0.021080,0.315403,NaN,5.958538,NaN,16.400000,27.031030,33.400002
4,010Q650 X 2020-01-06,2020-01-06,010Q650,21.0,9.0,52.0,1164.82,95,13.900001,44.700001,...,0.000171,0.020677,0.022017,0.307463,0.315403,5.755760,5.958538,6.911948,23.971857,21.300001


In [13]:
for num in range(len(test)):
    if num%94 == 0:
        test['precipitable_water_entire_atmosphere_yday_'][num] = np.NaN
for num in range(len(train)):
    if num%94 == 0:
        train['precipitable_water_entire_atmosphere_yday_'][num] = np.NaN

C:\Users\E-calculus\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\E-calculus\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
for num in range(len(test)):
    if num%94 == 0:
        test['temperature_2m_above_ground_yday_'][num] = np.NaN
for num in range(len(train)):
    if num%94 == 0:
        train['temperature_2m_above_ground_yday_'][num] = np.NaN

C:\Users\E-calculus\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\E-calculus\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
for num in range(len(test)):
    if num%94 == 0:
        test['relative_humidity_2m_above_ground_yday_'][num] = np.NaN
for num in range(len(train)):
    if num%94 == 0:
        train['relative_humidity_2m_above_ground_yday_'][num] = np.NaN

C:\Users\E-calculus\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\E-calculus\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
train.drop(['L3_HCHO_sensor_azimuth_angle','L3_HCHO_sensor_zenith_angle','L3_HCHO_solar_azimuth_angle','L3_HCHO_solar_zenith_angle','L3_CO_sensor_zenith_angle','L3_CO_solar_zenith_angle','L3_SO2_sensor_azimuth_angle','L3_SO2_sensor_zenith_angle','L3_SO2_solar_azimuth_angle','L3_SO2_solar_zenith_angle'], axis = 1, inplace = True)

In [17]:
test.drop(['L3_HCHO_sensor_azimuth_angle','L3_HCHO_sensor_zenith_angle','L3_HCHO_solar_azimuth_angle','L3_HCHO_solar_zenith_angle','L3_CO_sensor_zenith_angle','L3_CO_solar_zenith_angle','L3_SO2_sensor_azimuth_angle','L3_SO2_sensor_zenith_angle','L3_SO2_solar_azimuth_angle','L3_SO2_solar_zenith_angle'], axis = 1, inplace = True)

In [18]:
def drop_columns(data_, data_row_):
    col = []
    pct_ = 0.7
    for i in data_:
        if data_[i].isnull().sum() >= (pct_ * data_row_):
            col.append(i)
    return col

In [19]:
print(len(drop_columns(train,30557)),drop_columns(train,30557))

7 ['L3_CH4_CH4_column_volume_mixing_ratio_dry_air', 'L3_CH4_aerosol_height', 'L3_CH4_aerosol_optical_depth', 'L3_CH4_sensor_azimuth_angle', 'L3_CH4_sensor_zenith_angle', 'L3_CH4_solar_azimuth_angle', 'L3_CH4_solar_zenith_angle']


In [20]:
print(len(drop_columns(test,16136)),drop_columns(test,16136))

7 ['L3_CH4_CH4_column_volume_mixing_ratio_dry_air', 'L3_CH4_aerosol_height', 'L3_CH4_aerosol_optical_depth', 'L3_CH4_sensor_azimuth_angle', 'L3_CH4_sensor_zenith_angle', 'L3_CH4_solar_azimuth_angle', 'L3_CH4_solar_zenith_angle']


In [21]:
train.drop(drop_columns(train,30557),axis = 1, inplace = True)
test.drop(drop_columns(test,16136),axis = 1, inplace = True)

In [22]:
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

In [38]:
train["Date"] = pd.to_datetime(train['Date'])
test["Date"] = pd.to_datetime(test['Date'])

def func3(row):
    if row.dayofweek == 5 or row.dayofweek == 6:
        return 1
    else:
        return 0 






def date_(df):
      
   df['Week days']=df['Date'].dt.dayofweek
   df['Weekend'] = df['Date'].apply(func3)
   df['month']=0
   df['day']=0
   df['winter']=0
   for i in range(len(df)):  
           
           df['month'][i]=df['Date'][i].month 
           df['day'][i]=df['Date'][i].day
           if (df['month'][i]==1 or df['month'][i]==2 ):
             df['winter'][i]=1

date_(train)
date_(test)

C:\Users\E-calculus\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\E-calculus\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\E-calculus\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [ ]:
train["Date"] = pd.to_datetime(train['Date'])
test["Date"] = pd.to_datetime(test['Date'])

def func3(row):
    if row.dayofweek == 5 or row.dayofweek == 6:
        return 1
    else:
        return 0 

def date_split_(df):
    df['Week days'] = df['Date'].dt.dayofweek
    df['Week days'] = df['Date'].apply(func3)
    df['month'] = 0
    df['day'] = 0
    df['Winter'] = 0
    for i in range(len(df)):
        df['month'][i]=df['Date'][i].month
        df['day'][i]=df['Date'][i].day
        if (df['month'][i]==1 or df['month'][i]==2 )
            df['winter'][i]=1
date_split_(train)
date_split_(test)


In [ ]:
train=train.fillna(method='ffill')
train=train.fillna(train.mean())
test=test.fillna(method ='ffill')
test=test.fillna(test.mean())

In [ ]:
def func4(df):
    temperature_mean = pd.DataFrame(df.groupby(['Place_ID','month'])['temperature_2m_above_ground'].mean())
    df['mean_temperature_per_month']=0
    for i in range(len(df))
        df['mean_temperature_per_month'][i] = temperature_mean.loc[(df['Place_ID'][i], df['month'][i]),:]

In [ ]:
func4(train), func4(test)

In [ ]:
pca = PCA(random_state=42,n_components=1)
L3_AER_family_train =  train.filter(regex='L3_AER_.*')
train_pca = pca.fit_transform(L3_AER_family)
train['pca_AER_'] = train_pca[:,0]
L3_AER_family_test =  test.filter(regex='L3_AER.*')
test_pca = pca.transform(pg_features)
test['pca_AER_'] = test_pca[:,0]

pca = PCA(random_state=42,n_components=1)

L3_NO2_family_train =  train.filter(regex='L3_NO2_.*')

train_pca = pca.fit_transform(L3_NO2_family_train)

train['pca_NO2_'] = train_pca[:,0]

L3_NO2_family_test =  test.filter(regex='L3_NO2.*')

test_pca = pca.transform(L3_NO2_family_test)

test['pca_NO2_'] = test_pca[:,0]

In [ ]:
trainag=train.copy()
testag=test.copy()

In [ ]:
train.drop(['Place_ID X Date','Date','Place_ID'],inplace=True,axis=1)
test.drop(['Place_ID X Date','Date','Place_ID'],inplace=True,axis=1)
target=train['target']
train.drop('target',axis=1,inplace=True)

In [ ]:
feature=['precipitable_water_entire_atmosphere','relative_humidity_2m_above_ground','specific_humidity_2m_above_ground','temperature_2m_above_ground', 'u_component_of_wind_10m_above_ground', 'v_component_of_wind_10m_above_ground', 	'L3_NO2_NO2_column_number_density', 'L3_SO2_SO2_column_number_density','L3_SO2_SO2_column_number_density_amf','L3_SO2_SO2_slant_column_number_density','L3_NO2_tropospheric_NO2_column_number_density', 'L3_HCHO_tropospheric_HCHO_column_number_density', 'L3_CO_CO_column_number_density', 'L3_HCHO_HCHO_slant_column_number_density'	,'L3_CO_H2O_column_number_density','L3_O3_O3_column_number_density'	,'L3_NO2_NO2_slant_column_number_density']
train1=train.copy()
test1=test.copy()
for c in list(feature):
  if c not in l:
    train1.drop(c,inplace=True,axis=1)
    test1.drop(c,inplace=True,axis=1)
    
pca = PCA(random_state=42,n_components=1)
train_pca = pca.fit_transform(train1)
train['pca_feature'] = train_pca[:,0]
test_pca = pca.transform(test1)
test['pca_feature'] = test_pca[:,0]

In [ ]:
train1=train.copy()
test1=test.copy()

In [ ]:
pca = PCA(random_state=42,n_components=1)
train_pca = pca.fit_transform(train1)
train['pca_feature'] = train_pca[:,0]
test_pca = pca.transform(test1)
test['pca_feature'] = test_pca[:,0]

In [ ]:
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
X_train,X_test,y_train,y_test= train_test_split(train, target, test_size = 0.15, random_state = 42)

In [ ]:
from catboost import CatBoostRegressor
cb_model = CatBoostRegressor(iterations=30000,
                             learning_rate=0.045,
                             depth=8,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=300)
cb_model.fit(train,target,
             use_best_model=True,
             verbose=50)

In [ ]:
pred_test_cat2 = (cb_model.predict(test))

In [ ]:
import lightgbm as lgb

In [ ]:
boost_params = {'eval_metric': 'rmse'}
lgb_ = lgb.LGBMRegressor(
max_depth=30,
learning_rate=0.0,
n_estimators=2000,
objective='regression',
gamma=0,
min_child_weight=1,
subsample=1,
colsample_bytree=1,
scale_pos_weight=1,
seed=42,random_state = 42,
**boost_params)
lgb_.fit(dev_X,dev_y)
accuracyxgboost = round(lgb_.score(val_X, val_y)*100,2)
predict_xgboost = lgb_.predict(val_X)
msexgboost = mean_squared_error(val_y,predict_xgboost)
rmsexgboost= np.sqrt(msexgboost)
with open('lb1.pickle','wb') as f:
    pickle.dump(lgb_,f)
print(rmsexgboost)
print(accuracyxgboost)


In [32]:
features_corr_ = ['L3_NO2_NO2_column_number_density','L3_HCHO_tropospheric_HCHO_column_number_density','L3_CO_CO_column_number_density','L3_CLOUD_surface_albedo','L3_CLOUD_cloud_optical_depth']

for i in features_corr_:
    train[i+'_new_'+str(2)] = train[i].shift(periods = 2)
    train[i+'_new_'+str(3)] = train[i].shift(periods = 3)
    test[i+'_new_'+str(2)] = test[i].shift(periods = 2)
    test[i+'_new_'+str(3)] = test[i].shift(periods = 3)